<a href="https://colab.research.google.com/github/hwarang97/paperswithcode/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive/')

!pip install torchinfo
from torchinfo import summary

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

Mounted at /content/drive/
cuda


In [29]:
def select_model(depth):
    if depth == 18:
        return ResNet_18()

    elif depth == 34:
        return ResNet_34()

    elif depth == 50:
        return ResNet_50()

    elif depth == 101:
        return ResNet_101()

    elif depth == 152:
        return ResNet_152()

    else:
        print(f'check if model depth is in {[18, 34, 50, 101, 152]}')

In [41]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.block(x)
        return x

class Identity_Block(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.block = nn.Sequential(
            BasicConv2d(in_channels, in_channels),
            BasicConv2d(in_channels, in_channels)
        )

    def forward(self, x):
        x = self.block(x)
        return x

# class Identity_Block_L(nn.Module):
#     def __init__(self, in_channels):
#         super().__init__()
#         self.block = nn.Sequential(
#             BasicConv2d(in_channels, in_channels, kernel_size=1, padding=0),
#             BasicConv2d(in_channels, in_channels),
#             BasicConv2d(in_channels, 4*in_channels)
#         )

#     def forward(self, x):
#         x = self.block(x)
#         return x

class projection_Block(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.block = nn.Sequential(
            BasicConv2d(in_channels, 2*in_channels, stride=2),
            # nn.Conv2d(in_channels, 2*in_channels, kernel_size=3, stride=2, padding=1, bias=False),
            # nn.BatchNorm2d(2*in_channels),
            # nn.ReLU(),
            BasicConv2d(2*in_channels, 2*in_channels)
        )

    def forward(self, x):
        x = self.block(x)
        return x

# class projection_Block(nn.Module):
#     def __init__(self, in_channels):
#         super().__init__()
#         self.block = nn.Sequential(
#             nn.Conv2d(in_channels, 2*in_channels, kernel_size=3, stride=2, padding=1, bias=False),
#             nn.BatchNorm2d(2*in_channels),
#             nn.ReLU(),
#             BasicConv2d(2*in_channels, 2*in_channels)
#         )

#     def forward(self, x):
#         x = self.block(x)
#         return x

class ResNet_18(nn.Module):
    def __init__(self):
        super().__init__()
        self.channels = 64

        self.conv_2 = nn.Sequential(
            *self.make_layer(2, self.channels)
        )
        self.conv_3 = nn.Sequential(
            projection_Block(self.channels),
            *self.make_layer(1, 2*self.channels)
        )
        self.conv_4 = nn.Sequential(
            projection_Block(2*self.channels),
            *self.make_layer(1, 4*self.channels)
        )
        self.conv_5 = nn.Sequential(
            projection_Block(4*self.channels),
            *self.make_layer(1, 8*self.channels)
        )
        self.gap = nn.AvgPool2d(7)
        self.fc = nn.Linear(8*self.channels,1000)

    def make_layer(self, num, channels):
        layer = []
        for _ in range(num):
            layer.append(Identity_Block(channels))
        return layer

    def forward(self, x):
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.conv_4(x)
        x = self.conv_5(x)
        x = self.gap(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

class ResNet_34(nn.Module):
    def __init__(self):
        super().__init__()
        self.channels = 64

        self.conv_2 = nn.Sequential(
            *self.make_layer(3, self.channels)
        )
        self.conv_3 = nn.Sequential(
            projection_Block(self.channels),
            *self.make_layer(3, 2*self.channels)
        )
        self.conv_4 = nn.Sequential(
            projection_Block(2*self.channels),
            *self.make_layer(5, 4*self.channels)
        )
        self.conv_5 = nn.Sequential(
            projection_Block(4*self.channels),
            *self.make_layer(2, 8*self.channels)
        )
        self.gap = nn.AvgPool2d(7)
        self.fc = nn.Linear(8*self.channels,1000)

    def make_layer(self, num, channels):
        layer = []
        for _ in range(num):
            layer.append(Identity_Block(channels))
        return layer

    def forward(self, x):
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.conv_4(x)
        x = self.conv_5(x)
        x = self.gap(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

In [42]:
depth = 18 # [18, 34, 50, 101, 152]
model = select_model(depth)
model.train()

summary(model, input_size=(10,64,56,56))

Layer (type:depth-idx)                             Output Shape              Param #
ResNet_18                                          [10, 1000]                --
├─Sequential: 1-1                                  [10, 64, 56, 56]          --
│    └─Identity_Block: 2-1                         [10, 64, 56, 56]          --
│    │    └─Sequential: 3-1                        [10, 64, 56, 56]          73,984
│    └─Identity_Block: 2-2                         [10, 64, 56, 56]          --
│    │    └─Sequential: 3-2                        [10, 64, 56, 56]          73,984
├─Sequential: 1-2                                  [10, 128, 28, 28]         --
│    └─projection_Block: 2-3                       [10, 128, 28, 28]         --
│    │    └─Sequential: 3-3                        [10, 128, 28, 28]         221,696
│    └─Identity_Block: 2-4                         [10, 128, 28, 28]         --
│    │    └─Sequential: 3-4                        [10, 128, 28, 28]         295,424
├─Sequential: 1-3